In [3]:
# only for gated models 
from huggingface_hub import login
login()

In [1]:
import sys
import os
import subprocess

print("Python executable:", sys.executable)
print("Python prefix:", sys.prefix)
print("Conda environment:", os.environ.get('CONDA_DEFAULT_ENV'))
print("Conda prefix:", os.environ.get('CONDA_PREFIX'))

# Check pip location
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Pip location:", result.stdout.strip())

# Check pip version
result = subprocess.run(['pip', '--version'], capture_output=True, text=True)
print("Pip version:", result.stdout.strip())

Python executable: /home/sagemaker-user/.conda/envs/inception/bin/python
Python prefix: /home/sagemaker-user/.conda/envs/inception
Conda environment: base
Conda prefix: /opt/conda
Pip location: /opt/conda/bin/pip
Pip version: pip 25.1.1 from /opt/conda/lib/python3.12/site-packages/pip (python 3.12)


In [2]:
import os
import sys

# Add environment's bin directory to PATH
env_bin = os.path.dirname(sys.executable)
current_path = os.environ['PATH']
if env_bin not in current_path:
    os.environ['PATH'] = f"{env_bin}:{current_path}"

# Verify fix
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Fixed pip location:", result.stdout.strip())

Fixed pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip


In [7]:
# -*- coding: utf-8 -*-

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "inceptionai/jais-family-590m-chat"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)


def get_response(text, tokenizer=tokenizer, model=model):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    inputs = input_ids.to(device)
    input_len = inputs.shape[-1]
    generate_ids = model.generate(
        inputs,
        top_p=0.9,
        temperature=0.7,
        max_length=2048,
        min_length=input_len + 4,
        repetition_penalty=1.5,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,  # Add this line
        use_cache=False,  # Add this line
        no_repeat_ngram_size=3  # Prevent 3-gram repetition
    )
    response = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]
    return response


text = "عاصمة دولة الإمارات العربية المتحدة ه"
print(get_response(text))

text = "The capital of UAE is"
print(get_response(text))

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration_jais.py: 0.00B [00:00, ?B/s]

...

modeling_jais.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/inceptionai/jais-family-590m-chat:
- modeling_jais.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00001.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

عاصمة دولة الإمارات العربية المتحدة هيفيسيديديديريدياري ديابولي دبسول ديسيلديبدبريل ديلليديب دوبل ليبه درهيبيل دوبروي.
The capital of UAE is in the city...


In [10]:
text = "How are UAE and INdia relations?"
print(get_response(text))

How are UAE and INdia relations?**|>||<<>>€||<<2.1.3


# 7 Billion Adapted Model - Jias

In [5]:
# -*- coding: utf-8 -*-
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "MBZUAI/Llama-3-Nanda-10B-Chat"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True, offload_folder="./offload" )


prompt_hindi = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>{Question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# After loading tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token   # or use a different token

# In your generation function
def get_response(text, tokenizer=tokenizer, model=model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    generate_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Add this
        pad_token_id=tokenizer.pad_token_id,  # Add this
        max_length=2048,
        do_sample=True,
        temperature=0.3
    )
    response = tokenizer.decode(generate_ids[0], skip_special_tokens=True)
    return response


ques= "मुझे यूएई के बारे में कुछ रोचक तथ्य बताएं?"
text = prompt_hindi.format_map({'Question':ques})
print(get_response(text))


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/sagemaker-user/.conda/envs/inception/lib/python3.9/site-packages/transformers/generation/utils.py:2412: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "inceptionai/jais-family-256m-chat"

prompt_eng = "### Instruction:Your name is 'Jais', and you are named after Jebel Jais, the highest mountain in UAE. You were made by 'Inception' in the UAE. You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Complete the conversation between [|Human|] and [|AI|]:\n### Input: [|Human|] {Question}\n[|AI|]\n### Response :"
prompt_ar = "### Instruction:اسمك \"جيس\" وسميت على اسم جبل جيس اعلى جبل في الامارات. تم بنائك بواسطة Inception في الإمارات. أنت مساعد مفيد ومحترم وصادق. أجب دائمًا بأكبر قدر ممكن من المساعدة، مع الحفاظ على البقاء أمناً. أكمل المحادثة بين [|Human|] و[|AI|] :\n### Input:[|Human|] {Question}\n[|AI|]\n### Response :"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)


def get_response(text, tokenizer=tokenizer, model=model):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    inputs = input_ids.to(device)
    input_len = inputs.shape[-1]
    generate_ids = model.generate(
        inputs,
        top_p=0.9,
        temperature=0.3,
        max_length=2048,
        min_length=input_len + 4,
        repetition_penalty=1.2,
        do_sample=True,
    )
    response = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]
    response = response.split("### Response :")[-1]
    return response


ques = "ما هي عاصمة الامارات؟"
text = prompt_ar.format_map({'Question': ques})
print(get_response(text))

ques = "What is the capital of UAE?"
text = prompt_eng.format_map({'Question': ques})
print(get_response(text))


In [5]:
!pwd

/home/sagemaker-user/Inception
